In [3]:
import pandas as pd
import os, sys, random
module_path = os.path.abspath(os.path.join('../src/Sentiment-Analysis-Twitter/'))
if module_path not in sys.path:
    sys.path.append(module_path)

import nltk, re
import collections

import time
import sanderstwitter02
import stanfordcorpus
import stats
from __future__ import print_function

BASE_DIR = '../src/Sentiment-Analysis-Twitter/'

In [4]:
tweets1 = sanderstwitter02.getTweetsRawData(
    BASE_DIR + 'sentiment.csv')
tweets2 = stanfordcorpus.getNormalisedTweets(
    BASE_DIR + 'stanfordcorpus/'+stanfordcorpus.FULLDATA+'.5000.norm.csv')
#random.shuffle(tweets1)
#random.shuffle(tweets2)
tweets = tweets1 + tweets2
random.shuffle( tweets )
#tweets = tweets[:100]
print('\nlen( tweets ) = '+str(len( tweets )))


len( tweets ) = 10111


In [5]:
tweets[42]

['is with giordana dying of hunger.. but we have no money... ',
 'neg',
 'NO_QUERY',
 []]

In [6]:
from functools import wraps
import preprocessing

procTweets = [ (preprocessing.processAll(text, subject=subj, query=quer), sent) 
              for (text, sent, subj, quer) in tweets]

In [7]:
stemmer = nltk.stem.PorterStemmer()

all_tweets = []                                             #DATADICT: all_tweets =   [ (words, sentiment), ... ]
for (text, sentiment) in procTweets:
    words = [word if(word[0:2]=='__') else word.lower() \
                for word in text.split() \
                if len(word) >= 3]
    words = [stemmer.stem(w) for w in words]                #DATADICT: words = [ 'word1', 'word2', ... ]
    all_tweets.append((words, sentiment))

In [8]:
train_tweets = all_tweets[:10000]
test_tweets = all_tweets[10001:]

In [9]:
add_ngram_feat = 2

unigrams_fd = nltk.FreqDist()
if add_ngram_feat > 1 :
    n_grams_fd = nltk.FreqDist()

for( words, sentiment ) in train_tweets:
    words_uni = words
    unigrams_fd.update(words)

    if add_ngram_feat>=2 :
        words_bi  = [ ','.join(map(str,bg)) for bg in nltk.bigrams(words) ]
        n_grams_fd.update( words_bi )

    if add_ngram_feat>=3 :
        words_tri  = [ ','.join(map(str,tg)) for tg in nltk.trigrams(words) ]
        n_grams_fd.update( words_tri )

In [10]:
unigrams_sorted = unigrams_fd.keys()
#bigrams_sorted = nltk.FreqDist(bigrams).keys()
#trigrams_sorted = nltk.FreqDist(trigrams).keys()
if add_ngram_feat > 1 :
    print( '\nlen( n_grams ) = '+str(len( n_grams_fd )) )
    ngrams_sorted = [ k for (k,v) in n_grams_fd.items() if v>1]
    print( '\nlen( ngrams_sorted ) = '+str(len( ngrams_sorted )) )


len( n_grams ) = 64426

len( ngrams_sorted ) = 12415


In [11]:
def get_word_features(words):
    bag = {}
    words_uni = [ 'has(%s)'% ug for ug in words ]

    if( add_ngram_feat>=2 ):
        words_bi  = [ 'has(%s)'% ','.join(map(str,bg)) for bg in nltk.bigrams(words) ]
    else:
        words_bi  = []

    if( add_ngram_feat>=3 ):
        words_tri = [ 'has(%s)'% ','.join(map(str,tg)) for tg in nltk.trigrams(words) ]
    else:
        words_tri = []

    for f in words_uni+words_bi+words_tri:
        bag[f] = 1

    #bag = collections.Counter(words_uni+words_bi+words_tri)
    return bag

In [12]:
word_features = [get_word_features(word[0]) for word in train_tweets]

In [13]:
negtn_regex = re.compile( r"""(?:
        ^(?:never|no|nothing|nowhere|noone|none|not|
            havent|hasnt|hadnt|cant|couldnt|shouldnt|
            wont|wouldnt|dont|doesnt|didnt|isnt|arent|aint
        )$
    )
    |
    n't
    """, re.X) 
def get_negation_features(words):
    INF = 0.0
    negtn = [ bool(negtn_regex.search(w)) for w in words ]

    left = [0.0] * len(words)
    prev = 0.0
    for i in range(0,len(words)):
        if( negtn[i] ):
            prev = 1.0
        left[i] = prev
        prev = max( 0.0, prev-0.1)

    right = [0.0] * len(words)
    prev = 0.0
    for i in reversed(range(0,len(words))):
        if( negtn[i] ):
            prev = 1.0
        right[i] = prev
        prev = max( 0.0, prev-0.1)

    return dict( zip(
                    ['neg_l('+w+')' for w in  words] + ['neg_r('+w+')' for w in  words],
                    left + right ) )

In [14]:
 def extract_features(words):
    features = {}

    word_features = get_word_features(words)
    features.update( word_features )

    negation_features = get_negation_features(words)
    features.update( negation_features )
    return features


In [15]:
v_train = nltk.classify.apply_features(extract_features, train_tweets)
v_test = nltk.classify.apply_features(extract_features, test_tweets)

In [16]:
v_test[2]

({'has(__hndl)': 1,
  'has(__hndl,how)': 1,
  u'has(__punc_qu)': 1,
  u'has(almighti)': 1,
  'has(almighti,__hndl)': 1,
  u'has(awesom)': 1,
  'has(awesom,__punc_qu)': 1,
  'has(from)': 1,
  'has(from,the)': 1,
  'has(got)': 1,
  'has(got,repli)': 1,
  'has(how)': 1,
  'has(how,awesom)': 1,
  u'has(repli)': 1,
  'has(repli,from)': 1,
  'has(the)': 1,
  'has(the,almighti)': 1,
  'has(woop)': 1,
  'has(woop,got)': 1,
  'neg_l(__hndl)': 0.0,
  u'neg_l(__punc_qu)': 0.0,
  u'neg_l(almighti)': 0.0,
  u'neg_l(awesom)': 0.0,
  'neg_l(from)': 0.0,
  'neg_l(got)': 0.0,
  'neg_l(how)': 0.0,
  u'neg_l(repli)': 0.0,
  'neg_l(the)': 0.0,
  'neg_l(woop)': 0.0,
  'neg_r(__hndl)': 0.0,
  u'neg_r(__punc_qu)': 0.0,
  u'neg_r(almighti)': 0.0,
  u'neg_r(awesom)': 0.0,
  'neg_r(from)': 0.0,
  'neg_r(got)': 0.0,
  'neg_r(how)': 0.0,
  u'neg_r(repli)': 0.0,
  'neg_r(the)': 0.0,
  'neg_r(woop)': 0.0},
 'pos')

In [17]:
model = nltk.classify.NaiveBayesClassifier.train(v_train)

In [18]:
nltk.classify.accuracy(model, v_test)

0.8454545454545455

In [19]:
model.classify(v_test[2][0])

'pos'

In [20]:
s = "I just ate"
stemmer = nltk.stem.PorterStemmer()
s = [stemmer.stem(w) for w in s.split()]

In [21]:
unigrams_fd = nltk.FreqDist()
unigrams_fd.update(s)

fs = extract_features(s)

In [22]:
model.classify(fs)

'neg'